In [ ]:
%matplotlib notebook

# https://en.wikipedia.org/wiki/Bessel_filter
# https://www.recordingblogs.com/wiki/bessel-filter
# https://www.recordingblogs.com/wiki/bilinear-transformation

import math
import sympy

import numpy as np
import matplotlib.pyplot as plt

s, z, Z = sympy.symbols('s z Z')

w0, wb, fc, fs = sympy.symbols('omega_0 omega_b F_c F_s')
# F_s sampling frequency
# F_c cutting frequency

### Second order butterworth with bilinear transform

In [ ]:
a0, a1, a2, b0, b1, b2 = sympy.symbols('a_0 a_1 a_2 b_0 b_1 b_2')

In [ ]:
coef = {
    'a_0' : w0**2,
    'a_1' : 2*w0**2,
    'a_2' : w0**2,
    'b_0' : 4 - 4*w0*sympy.cos(3*sympy.pi/4) + w0**2,
    'b_1' : 2*w0**2 - 8,
    'b_2' : 4 + 4*w0*sympy.cos(3*sympy.pi/4) + w0**2,
}

In [ ]:
H_low_2 = (a0 + a1*Z + a2*Z**2) / (b0 + b1*Z + b2*Z**2)
n, d = H_low_2.as_numer_denom()
H_low_2 = (n / b0).expand() / (d / b0).expand()
#H_low_2 = ((a0 + a1*Z + a2*Z**2) / b0) * (b0 / (b0 + b1*Z + b2*Z**2))

H_low_2

In [ ]:
cutoff = {"omega_0" : 1.0}#2 * math.tan(0.6 / 2)

In [ ]:
for c in coef :
    print(f"{c} = {float((coef[c] / coef['b_0']).subs(coef).subs(cutoff))}")

In [ ]:
class IIR_direct_form_I_2nd_order() :
    
    order = 2
    
    def __init__(self, a0, a1, a2, b0, b1, b2, x_ini=0.0, y_ini=0.0) :
        self.x = [x_ini,] * (self.order + 1)
        self.y = [y_ini,] * (self.order + 1)
        
        self.a0, self.a1, self.a2, self.b1, self.b2 = a0 / b0, a1 / b0, a2 / b0, b1 / b0, b2 / b0
        
        print(f"{self.a0} z0 + {self.a1} z1 + {self.a2} z2")
        print(f"1 + {self.b1} z1 + {self.b2} z2")

    def unwrap_360(self, x0, x1) :
        d = x0 - x1
        w = math.copysign(360.0, d) if abs(d) > 180.0 else 0.0
        
        return w
        
    def run(self, x0) :
        a0, a1, a2, b1, b2 = self.a0, self.a1, self.a2, self.b1, self.b2
        
        self.x = [x0 % 360.0,] + self.x[:-1]
        
        x1, x2 = self.x[:-1]
        y1, y2 = self.y[:-1]
                
        y = a0 * x0 + a1 * x1 + a2 * x2 - (
            b1 * y1 + b2 * y1
        )
        
        y0 = a0 * x0 + (1-a0) * y1
        
        self.y = [y0,] + self.y[:-1]
        
        print(x0, self.x, self.y, y0)
                
        return self.y[0]
    
    def run_wrapped_360(self, x0) :
        a0, a1, a2, b1, b2 = self.a0, self.a1, self.a2, self.b1, self.b2
        
        # for old elements, an unwrap is applied first
        w = self.unwrap_360(x0, self.x[0])
        self.x[1:] = [w + x for x in self.x[:-1]]
        self.y[1:] = [w + y for y in self.y[:-1]]
        
        self.x[0] = x0 % 360.0
        
        x0, x1, x2 = self.x
        _, y1, y2 = self.y
        
        y0 = a0 * x0 + a1 * x1 + a2 * x2 - (
            b1 * y1 + b2 * y1
        )
        
        y0 = (a0 * x0 + (1-a0) * y1)
        
        self.y[0] = y0
                        
        return self.y[0] % 360.0

In [ ]:
BW = IIR_direct_form_I_2nd_order(1/7, 0.0, 0.0, 1.0, 0.0, 0.0, x_ini=30.0, y_ini=30.0)

x_arr = np.array([30.0,] * 5 + [360.0 - 30.0,] * 20 + [30.0,] * 20)

y_arr = np.array([BW.run_wrapped_360(x) for x in x_arr])

plt.plot(x_arr)
plt.plot(y_arr, '+-')
plt.grid()
plt.show()

In [ ]:
BW = IIR_direct_form_I_2nd_order(0.666, 0.0, 0.0, 1.0, 0.0, 0.0)

x_arr = np.array([30.0,] * 5 + [-30.0,] * 10 + [30.0,] * 10)
y_arr = np.array([BW.run(x) for x in x_arr])

plt.plot(x_arr)
plt.plot(y_arr, '+-')
plt.grid()
plt.show()